In [22]:
#images taken from https://unsplash.com/wallpapers
from random import randrange
images = ["1.jpg"]
totalParity=0
totalHamming=0
dataSize= {"parity":7,"hamming":11}
#dataSize= {"parity":7,"hamming":4}
data = {"packet" : [] , "ptype" :[], "label" : []}

def createParity(bitstr,parity="even"):
    
    binaryCount = bitstr.count("1")
    if parity=="even":
        if binaryCount %2 ==0 : 
            returnValue = bitstr+"0"
        else :
            returnValue = bitstr+"1"
    else:
        # odd parity
        if binaryCount %2 ==1:
            returnValue = bitstr+"0"
        else : 
            returnValue = bitstr+"1"
    return returnValue

def createHamming(bitstr):
    #iam going to use hamming 15 11 means 4 bits parity 1 2 4 8 bits parity bits total bits are 
    '''
        p1 = 1 3 5 7 9 11 13 15
        p2 = 2 3 6 7 10 11 14 15
        p3 = 4 5 6 7 12 13 14 15
        p4 = 8 9 10 11 12 13 14 15
    '''
    packet = list(bitstr)
    packet.insert(0,"p")
    packet.insert(1,"p")
    packet.insert(3,"p")
    packet.insert(7,"p")
    p1=""
    p2=""
    p3=""
    p4=""
    
    for i in range(1,16):
    #for i in range(1,8):
        if bin(i)[2:].zfill(4)[-1]=="1" and i!=1:
            p1+=packet[i-1]
        if bin(i)[2:].zfill(4)[-2]=="1" and i!=2:
            p2+=packet[i-1]
        if bin(i)[2:].zfill(4)[-3]=="1" and i!=4:
            p3+=packet[i-1]
        if bin(i)[2:].zfill(4)[-4]=="1" and i!=8:
            p4+=packet[i-1]
    packet[0] = "0" if p1.count("1")%2 ==0 else "1"
    packet[1] = "0" if p2.count("1")%2 ==0 else "1"
    packet[3] = "0" if p3.count("1")%2 ==0 else "1"
    packet[7] = "0" if p4.count("1")%2 ==0 else "1"
    return ''.join(packet)




for image in images : 
    dosyaStart=0
    f=open(image,"rb")
    file_bytes = f.read()
    dosyaStop = len(file_bytes)*8 # converting bytes to bits
    bits = ''.join(format(byte, '08b') for byte in file_bytes)
    while dosyaStart < dosyaStop :
        label=0
        #myRandom = randrange(1,3)
        myRandom=2
        if myRandom ==1:
            totalParity+=1
            packet = bits[dosyaStart:dosyaStart+dataSize["parity"]]
            if len(packet)<dataSize["parity"]:
                break;
            dosyaStart+=dataSize["parity"]
            #evenOrOdd = randrange(1,3)
            evenOrOdd = 2
            if evenOrOdd == 1:
                ptype="odd"
                #packet = createParity(packet,"odd") sonuc lineer olmadigi icin sorun cikariyor
                packet = createParity(packet,"even")
            else : 
                ptype="even"
                packet = createParity(packet)
            if totalParity%2==0:
                label=1
                corruptbit = randrange(0,7)
                bit = packet[corruptbit]
                if bit == "1":
                    bit="0"
                else:
                    bit="1"
                packet=packet[:corruptbit] + bit + packet[corruptbit+1:]

        else : 
            totalHamming+=1
            #hamming code
            ptype="hamming"
            packet = bits[dosyaStart:dosyaStart+dataSize["hamming"]]
            dosyaStart+=dataSize["hamming"]
            if len(packet)<dataSize["hamming"]:
                break;
            packet=createHamming(packet)
            datapackets = [2,4,5,6,8,9,10,11,12,13,14]
            #datapackets = [2,4,5,6]
            if totalHamming%3==1:
                label=1
                corruptbit=datapackets[randrange(0,len(datapackets))]
                bit=packet[corruptbit]
                if bit=="1":
                    bit="0"
                else:
                    bit="1"
                packet=packet[:corruptbit] + bit + packet[corruptbit+1:]

        data["packet"].append(packet)
        data["ptype"].append(ptype)
        data["label"].append(label)
    f.close()

'''    
    while True:
        byte = f.read(1)
        if not byte : 
            break
        bits = bin(ord(byte))[2:]  # Python 3: byte[0] da kullanılabilir
        print(bits)
    '''




'    \n    while True:\n        byte = f.read(1)\n        if not byte : \n            break\n        bits = bin(ord(byte))[2:]  # Python 3: byte[0] da kullanılabilir\n        print(bits)\n    '

In [23]:
import pandas as pd
df=pd.DataFrame(data)

In [24]:
df = df.sample(frac=1, random_state=34).reset_index(drop=True)
train_size = int(0.8 * len(df))
traindf = df[:train_size]
testdf  = df[train_size:]
X_train = traindf["packet"].tolist()  # .tolist() ile bit string listesi
Y_train = traindf["label"].tolist()
X_test  = testdf["packet"].tolist()
Y_test  = testdf["label"].tolist()

print("Label distribution:", pd.Series(Y_train).value_counts())
traindf.head(10)

Label distribution: 0    1424087
1     711677
Name: count, dtype: int64


,packet,ptype,label
0,110111010111011,hamming,0
1,001000001100000,hamming,0
2,011000101001110,hamming,0
3,001001010100101,hamming,1
4,100011011010001,hamming,1
5,001000111011101,hamming,0
6,111100001111000,hamming,0
7,101100100000010,hamming,1
8,100011111101101,hamming,0
9,001111011011101,hamming,0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ---------------- Positional Encoding ----------------


class PosEncoding(nn.Module):
    def __init__(self, d_model, max_len=15):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(100.0) / d_model))  
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        power_of_2_mask = torch.ones(max_len)
        for i in [0, 1, 3, 7]:  
            if i < max_len:
                power_of_2_mask[i] = 2.0  
        pe = pe * power_of_2_mask.unsqueeze(1)  
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)


# ---------------- Multi-Head Attention ----------------


class Attention(nn.Module):
    def __init__(self, d_model, num_heads, seq_len=15, pad_idx=9):
        super().__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.q = nn.Linear(d_model, d_model)
        self.k = nn.Linear(d_model, d_model)
        self.v = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
        self.pad_idx = pad_idx

        # Hamming(15,11) parity map
        self.parity_map = {
            0: [2,4,6,8,10,12,14],   # P1
            1: [2,5,6,9,10,13,14],   # P2
            3: [4,5,6,11,12,13,14],  # P3
            7: [8,9,10,11,12,13,14], # P4
        }
        mask = torch.zeros(seq_len, seq_len)
        for p, bits in self.parity_map.items():
            mask[p, bits] = 1
        for i in range(seq_len):
            mask[i,i] = 1
        self.register_buffer('hamming_mask', mask)

    def forward(self, x, mask=None):
        b, seq_len, d_model = x.size()

        Q = self.q(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v(x).view(b, seq_len, self.num_heads, self.head_dim).transpose(1,2)

        scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.head_dim)

        # Padding mask
        if mask is not None:
            pad_mask = mask.unsqueeze(1).unsqueeze(2).float()  # [B,1,1,L]
        else:
            pad_mask = torch.ones(b,1,1,seq_len, device=x.device)

        # Hamming mask
        hamming_mask = self.hamming_mask[:seq_len, :seq_len].unsqueeze(0).unsqueeze(1)  # [1,1,L,L]

        # Combine
        combined_mask = pad_mask * hamming_mask
        scores = scores.masked_fill(combined_mask == 0, float('-inf'))

        attn = F.softmax(scores, dim=-1)
        out = torch.matmul(attn, V)
        out = out.transpose(1,2).contiguous().view(b, seq_len, d_model)
        return self.out(out)



# ---------------- Feed Forward ----------------
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))


# ---------------- Encoder Block ----------------
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = Attention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, mask=None):
        x = self.norm1(x + self.dropout(self.attn(x, mask)))
        x = self.norm2(x + self.dropout(self.ff(x)))
        return x


# ---------------- Bit Transformer ----------------
class BitTransformerClassifier(nn.Module):
    def __init__(self, d_model=128, num_heads=8, d_ff=512, num_layers=8,
                 num_classes=2, max_len=15, pad_idx=9):
        super().__init__()
        self.pad_idx = pad_idx
        vocab_size = pad_idx + 1
        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_enc = PosEncoding(d_model, max_len)
        self.layers = nn.ModuleList([EncoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        mask = (x != self.pad_idx)
        x = self.embed(x)
        x = self.pos_enc(x)
        for layer in self.layers:
            x = layer(x, mask)
        x = self.norm(x)
        mask_float = mask.unsqueeze(-1).float()
        x = (x * mask_float).sum(dim=1) / mask_float.sum(dim=1).clamp(min=1e-9)
        return self.classifier(x)


# ---------------- Focal Loss ----------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        if alpha is not None:
            self.alpha = torch.tensor(alpha, dtype=torch.float)
        else:
            self.alpha = None

    def forward(self, inputs, targets):
        device = inputs.device
        probs = F.softmax(inputs, dim=1).clamp(min=1e-9)
        pt = probs.gather(1, targets.unsqueeze(1)).squeeze(1)
        log_pt = torch.log(pt)
        if self.alpha is not None:
            alpha_t = self.alpha.to(device)[targets]  # <-- self.alpha GPU'ya taşındı
        else:
            alpha_t = torch.ones_like(pt, device=device)
        loss = -alpha_t * ((1 - pt) ** self.gamma) * log_pt
        return loss.mean()


# ---------------- Dataset Preparation ----------------
def prepare_tensor(bit_list, pad_idx=9, max_len=15):
    x = []
    for s in bit_list:
        bits = [int(b) for b in str(s).strip()]
        if len(bits) < max_len:
            bits += [pad_idx] * (max_len - len(bits))
        else:
            bits = bits[-max_len:]
        x.append(bits)
    return torch.tensor(x, dtype=torch.long)


# ---------------- Training Setup ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


d_model = 64
num_heads = 8
d_ff = 256
num_layers = 6
epochs = 10
pad_idx = 9
max_len_input = 15
batch_size = 128

model = BitTransformerClassifier(d_model, num_heads, d_ff, num_layers,
                                 num_classes=2, max_len=max_len_input, pad_idx=pad_idx)
model.to(device)

# Class distribution
class_counts = Counter(Y_train)
total = sum(class_counts.values())
alpha_for_1 = 1 - (class_counts[1] / total)
print(f"Class distribution: {class_counts}, Focal alpha for class 1: {alpha_for_1:.4f}")

criterion = FocalLoss(alpha=[1-alpha_for_1, alpha_for_1], gamma=2.0)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
steps_per_epoch = max(1, len(X_train)//batch_size)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=3e-4, epochs=epochs, steps_per_epoch=steps_per_epoch
)

X_train_tensor = prepare_tensor(X_train, pad_idx=pad_idx, max_len=max_len_input)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)

best_loss = float('inf')

# ---------------- Training Loop ----------------
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for i in range(0, len(X_train_tensor), batch_size):
        x_batch = X_train_tensor[i:i+batch_size].to(device)
        y_batch = Y_train_tensor[i:i+batch_size].to(device)

        optimizer.zero_grad()
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() * x_batch.size(0)
        _, predicted = torch.max(logits, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

    epoch_loss = running_loss / len(X_train_tensor)
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.6f} | Acc: {acc:.3f} | "
          f"Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), "best_hamming_model.pt")
        print("Best model saved!")



Using device: cuda
Class distribution: Counter({0: 1424087, 1: 711677}), Focal alpha for class 1: 0.6668
Epoch 1/10 | Loss: 0.077171 | Acc: 0.501 | Prec: 0.334 | Rec: 0.500 | F1: 0.400 | LR: 0.000084
✅ Best model saved!
Epoch 2/10 | Loss: 0.002892 | Acc: 0.989 | Prec: 0.980 | Rec: 0.986 | F1: 0.983 | LR: 0.000228
✅ Best model saved!
Epoch 3/10 | Loss: 0.000336 | Acc: 1.000 | Prec: 0.999 | Rec: 1.000 | F1: 0.999 | LR: 0.000300
✅ Best model saved!
Epoch 4/10 | Loss: 0.004200 | Acc: 0.979 | Prec: 0.960 | Rec: 0.979 | F1: 0.969 | LR: 0.000285
Epoch 5/10 | Loss: 0.000492 | Acc: 0.999 | Prec: 0.999 | Rec: 0.998 | F1: 0.999 | LR: 0.000244
Epoch 6/10 | Loss: 0.000294 | Acc: 1.000 | Prec: 1.000 | Rec: 0.999 | F1: 0.999 | LR: 0.000183
✅ Best model saved!


KeyboardInterrupt: 

In [28]:
pad_idx = 9
max_len_input = 15

model = BitTransformerClassifier(
    d_model=64,
    num_heads=8,
    d_ff=256,
    num_layers=6,
    num_classes=2,
    max_len=max_len_input,
    pad_idx=pad_idx
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("best_hamming_model.pt", map_location=device))
model.to(device)
model.eval() 

C:\Users\ali\AppData\Local\Temp\ipykernel_23800\1683030758.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_hamming_model.pt", map

BitTransformerClassifier(
  (embed): Embedding(10, 64, padding_idx=9)
  (pos_enc): PosEncoding()
  (layers): ModuleList(
    (0-5): 6 x EncoderBlock(
      (attn): Attention(
        (q): Linear(in_features=64, out_features=64, bias=True)
        (k): Linear(in_features=64, out_features=64, bias=True)
        (v): Linear(in_features=64, out_features=64, bias=True)
        (out): Linear(in_features=64, out_features=64, bias=True)
      )
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ff): FeedForward(
        (fc1): Linear(in_features=64, out_features=256, bias=True)
        (fc2): Linear(in_features=256, out_features=64, bias=True)
      )
      (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (classifier): Linear(in_features=64, out_features=2, bias=True)
)

In [ ]:


#X_test  = testdf["packet"].tolist()
#Y_test  = testdf["label"].tolist()
'''X_test_tensor = prepare_tensor(X_test, pad_idx=pad_idx, max_len=max_len_input).to(device)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long).to(device)
with torch.no_grad():
    logits = model(X_test_tensor)
    preds = torch.argmax(logits, dim=1)

acc = (preds == Y_test_tensor).float().mean().item()
print(f"Accuracy: {acc:.3f}")'''

print(X_test[7])
print(Y_test[7])
print(X_test[7].split())
X_test_tensor = prepare_tensor(X_test[7].split(),pad_idx=pad_idx , max_len = max_len_input).to(device) 
Y_test_tensor = torch.tensor([Y_test[7]],dtype=torch.long , device=device)
print(X_test_tensor)
print(Y_test_tensor)


batchsize = 512
mean = []

with torch.no_grad():
    for i in range(0 , len(X_test),batchsize):
        
        X_test_tensor = prepare_tensor(X_test[i:i+batchsize],pad_idx=pad_idx , max_len = max_len_input).to(device) 
        Y_test_tensor = torch.tensor(Y_test[i:i+batchsize],dtype=torch.long , device=device)
        logit = model(X_test_tensor)
        pred = torch.argmax(logit,dim=1)
        acc = (pred==Y_test_tensor).float().mean().item()
        mean.append(acc)


111111100111000
1
['111111100111000']
tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]], device='cuda:0')
tensor([1], device='cuda:0')


In [62]:
import numpy as np

lastmean = np.mean(mean)
print (lastmean)

0.9999999999428527
